# Using `cb.cbcheck` to check mass and stiffness

This and other jupyter notebooks are available here: https://github.com/twmacro/pyyeti/tree/master/docs/tutorials.

First, we need a valid Hurty-Craig-Bampton model to work with. Specifically, we need:

* a-set mass and stiffness
* b-set "uset" table (see description in [op2.rdnas2cam](../modules/nastran/generated/pyyeti.nastran.op2.rdnas2cam.html#pyyeti.nastran.op2.rdnas2cam))
    * Note: this is only needed for statically indeterminate interfaces
* b-set partition vector (relative to a-set)

We'll use superelement 102 from the test directory: "tests/nas2cam_csuper".

Aside:

* "nas2cam" stood for Nastran-to-CAM ... CAM is now replaced with Python but the Nastran DMAP retains the old name.

.. image:: se102.png
<img src="./se102.png" />

---
First, do some imports:

In [ ]:
import numpy as np
from io import StringIO
from pyyeti import nastran, cb
from pyyeti.nastran import op4

Need path to data files:

In [ ]:
import os
import inspect
pth = os.path.dirname(inspect.getfile(cb))
pth = os.path.join(pth, '../tests')
pth = os.path.join(pth, 'nas2cam_csuper')

#### Load the mass and stiffness from the .op4 file

This loads the data into a dict:

In [ ]:
mk = op4.load(os.path.join(pth, 'inboard.op4'))
mk.keys()

In [ ]:
maa = mk['mxx'][0]
kaa = mk['kxx'][0]

#### Get the USET table
The USET table has the boundary DOF information (id, location, coordinate system). This is needed for superelements with an indeterminate interface. The [nastran](../modules/nastran.html) module has the function [asm2uset](../modules/nastran/generated/pyyeti.nastran.bulk.asm2uset.html#pyyeti.nastran.bulk.asm2uset) (from [nastran.bulk](../modules/nastran/bulk.html), actually) which is handy for forming the USET table from bulk data. 

In [ ]:
uset, coords, bset = nastran.asm2uset(os.path.join(pth, 'inboard.asm'))
# show the coordinates (which are in basic):
uset.loc[(slice(None), 1), :]

#### Form-index style b-set partition vector into a-set
We already have `bset`, which is a boolean partition vector for the b-set:

In [ ]:
bset

Convert to index-style for [cbcheck](../modules/generated/pyyeti.cb.cbcheck.html#pyyeti.cb.cbcheck):

In [ ]:
b = bset.nonzero()[0]
b

---
#### Run cbcheck

Write to a string so we can look at the output a section at a time. The `em_filt` option filters the effective mass table print to only modes with 2% or higher values.

In [ ]:
with StringIO() as f:
    chk = cb.cbcheck(f, maa, kaa, b, b[:6], uset, em_filt=2)
    output = f.getvalue().splitlines()
lines = output[:]

First, a note on *possible* output from [cbcheck](../modules/generated/pyyeti.cb.cbcheck.html#pyyeti.cb.cbcheck) about iterations and convergence. That is information from the "subspace iteration" eigensolver [pyyeti.ytools.eig_si](../modules/generated/pyyeti.ytools.eig_si.html#pyyeti.ytools.eig_si). That routine is called to clean up the lowest frequency modes that are computed by [scipy.la.eigh](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.eigh.html) -- which can be slightly off. That output is not produced for this case, since a more general eigensolver is called; this is because (as we'll see) the mass fails the positive-definite check and the stiffness fails the symmetry check. 

Before going through the text output of [cbcheck](../modules/generated/pyyeti.cb.cbcheck.html#pyyeti.cb.cbcheck), let's take a quick look at the SimpleNamespace that it returns:

In [ ]:
from pyyeti.pp import PP
PP(chk)

The SimpleNamespace contains the reordered and converted versions of the inputs, three different sets of rigid-body modes (stiffness-based, eigenvalue-based, and geometry-based), the Hurty-Craig-Bampton fixed base frequencies (Hz), and some modal effective mass tables (one in mass units, the other in percent of total). The documentation below will cover some of these items in more detail. The following shows the `.effmass_percent` DataFrame for this model. The first mode contains 31.9% of the mass in the 'T2' direction and 63.7% of the inertia about the 'R3' axis. It also has no mass moving in the 'T1' direction. You can compare the DataFrame shown to the values printed in the text output at the end of this tutorial.

Note: A zero modal effective mass value actually just means that the masses sum to zero in that direction for that mode; which is always the case for flexible (non-rigid) free-free modes.

In [ ]:
chk.effmass_percent

We'll now focus on the text output of [cbcheck](../modules/generated/pyyeti.cb.cbcheck.html#pyyeti.cb.cbcheck). First, we'll define a simple printing function for cleaner output viewing:

In [ ]:
def prt(lines, next_string):
    for n, line in enumerate(lines):
        if next_string in line:
            break
    else:
        n += 1
    print('\n'.join(lines[:n]))
    return lines[n:]

---
The first 13 lines contain summary information. In this case, we see a warning that the mass is not positive definite and the stiffness is not symmetric. This doesn't necessarily mean the model is bad; it could be that it's just a little off from perfect. Everything else is as it should be:

In [ ]:
lines = prt(lines, 'properly restrain')

Since the stiffness didn't pass the symmetry check, it's worthwhile to print a check value. Comparing this value to the maximum KBB value (above) shows that the stiffness is *very* close to symmetric:

In [ ]:
abs(kaa-kaa.T).max()

---
Next, for statically-indeterminate models, there is a check to see if the `bref` DOF properly restrain rigid-body motion. This is similar to the SUPORT card in Nastran.

For this model, the check passes. If it failed, it would say: "Check: FAIL. Assess values below before running CLA." instead of: "Check: PASS. Values printed below for reference.".

The only large percent differences are for numerical zeros.

In [ ]:
lines = prt(lines, 'Stiffness-based coordinates')

---
The next section shows coordinate location information as computed from the stiffness. This first node is the reference and the others are relative to that node (and in the coordinate system of that node). The largest coordinate location error is printed for inspection. Here, the error is very small.


In [ ]:
lines = prt(lines, 'RB Translation')

---
The next section shows checks displacements results from rigid-body motion. All three types of rigid-body modes are used. Here everything is 1.000, perfect.

In [ ]:
lines = prt(lines, 'MASS PROPERTIES CHECKS')

---
The next section prints the 3 6x6 mass matrices for inspection. Information derived from these is printed afterwards. Note that the geometry reference point is different from the other two (see the [cbcheck](../modules/generated/pyyeti.cb.cbcheck.html#pyyeti.cb.cbcheck) input). Since we didn't set it, the reference for the geometry-based rb modes is (0, 0, 0) in the basic system.

In [ ]:
lines = prt(lines, 'Comparisons from mass properties')

---
CG location comparison is next:

In [ ]:
lines = prt(lines, 'Radius of gyration')

---
Radius of gyration checks are next. The radius of gyration about an axis is the radius where all the mass would be if all the mass were at a single radius. These values should make sense with your structure. If a radius is beyond your dimensions for example, you know something is wrong (yes, this has happened ... due to a badly written CONM1 or CONM2 Nastran card).

In [ ]:
lines = prt(lines, 'Stiffness-based Inertia')

---
Inertia values are printed next for inspection:

In [ ]:
lines = prt(lines, 'GROUNDING CHECKS')

---
Grounding checks are next. This is likely the largest section. This model is very clean. Note that the grounding forces for the geometry-based rigid-body modes can only include the b-set while the other two include the q-set.

If the stiffness and eigenvalue based checks are good, but the geometry one is not good, it probably means you have a mistake in your USET table (bad coordinate system, incorrect grid location, or the grids are in the wrong order).

In [ ]:
lines = prt(lines, 'FREE-FREE MODES')

---
The free-free mode check is next. The rigid-body modes should be close to zero frequency. (I actually depend on this more than the grounding checks above to check for grounding.) As noted previously, this model is very clean.

In [ ]:
lines = prt(lines, 'Modal Effective Mass')

---
The last section prints the modal effective mass using the geometry-based rb modes. This should match the provided documentation if there is any.

In [ ]:
lines = prt(lines, 'end of file')